# IPyForceGraph 🚀

`ipyforcegraph` wraps [force-graph] and [3d-force-graph] for data-driven views of graphs
with up to a few thousand nodes with composable behaviors.

[3d-force-graph]: https://github.com/vasturiano/force-graph
[force-graph]: https://github.com/vasturiano/force-graph

In [ ]:
if __name__ == "__main__" and "pyodide" in __import__("sys").modules:
    %pip install -q -r requirements.txt

In [ ]:
import json
from pathlib import Path

import ipywidgets as W
import traitlets as T

from ipyforcegraph.forcegraph import DataFrameSource, ForceGraph

## A Simple Example

In [ ]:
def make_a_simple_example(
    source=None, dataset="datasets/miserables.json", GraphClass=ForceGraph
):
    if source is None:
        data = json.loads(Path(dataset).read_text())
        source = DataFrameSource(**data)
    fg = GraphClass(source=source, layout=dict(min_height="500px"))
    style = W.HTML(
        """<style>
        .jp-fg-demo{
            --jp-widgets-container-padding: 0.25em;
            --jp-widgets-inline-width: auto;
        } 
        .widget-box:empty{display:none;}
    </style>"""
    )
    graph_ui = W.VBox()
    node_ui = W.VBox()
    link_ui = W.VBox()
    ui = W.VBox(
        [graph_ui, node_ui, link_ui], layout=dict(width="500px", overflow_y="scroll")
    )
    box = W.HBox([style, fg, ui], layout=dict(height="100%", max_height="100vh"))
    box.add_class("jp-fg-demo")
    box.add_traits(
        behaviors=T.Dict(),
        node_ui=T.Dict(),
        link_ui=T.Dict(),
        graph_ui=T.Dict(),
    )

    def on_box_behaviors(change=None):
        new_behaviors = [b for b in box.behaviors.values() if b is not None]
        fg.behaviors = tuple(new_behaviors)

    box.observe(on_box_behaviors, ["behaviors"])

    T.dlink((box, "graph_ui"), (graph_ui, "children"), lambda d: tuple(d.values()))
    T.dlink((box, "node_ui"), (node_ui, "children"), lambda d: tuple(d.values()))
    T.dlink((box, "link_ui"), (link_ui, "children"), lambda d: tuple(d.values()))
    return fg, box

In [ ]:
if __name__ == "__main__":
    fg, box = make_a_simple_example()
    display(box)

## Data Formats

Some sample data in various formats are included. The simplest form expects two
`DataFrame`-like structures:

- `nodes` should contain an `id` field of type `str` or `int`
  - if missing, the node's row index in the columns will be used
- `links` should contain `source` and `target` columns, and may refer to either the `id`
  or row index

```{hint}
Many of the graph [behaviors](./Behaviors.ipynb) reference `column_name`s from these fields,
or can use templates to construct values from multiple fields.
```

# Read More

## Examples

- [👟 Behaviors](./Behaviors.ipynb)
- [🔮 3D](./3D.ipynb)

## Miscellaneous

- [🔧 Demo Utilities](./Utils.ipynb)